In [ ]:
!pip install datasets
!pip install huggingface_hub

Need to log in to an account that has access to Q72



Read Token

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from datasets import load_dataset, concatenate_datasets

In [ ]:
q72 = load_dataset("nyu-dice-lab/allenai_WildChat-1M-Full-Qwen_Qwen2.5-72B-Instruct")
mmlu = load_dataset("cais/mmlu", "auxiliary_train")
tulu3 = load_dataset("allenai/tulu-3-sft-personas-algebra")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
q72 = q72.remove_columns(['conversation_hash']).rename_column('conversation', 'prompt').rename_column('model', 'response')['train']

In [ ]:
tulu3 = tulu3.remove_columns(['id']).rename_column('messages', 'response')['train']

In [ ]:
mmlu = mmlu.rename_column('train', 'prompt')['train'].add_column('response', range(len(mmlu['train'])))

In [ ]:
q72 = q72.shuffle(seed=42).select(range(200000))

In [ ]:
q72, mmlu, tulu3

(Dataset({
     features: ['prompt', 'response'],
     num_rows: 200000
 }),
 Dataset({
     features: ['prompt', 'response'],
     num_rows: 99842
 }),
 Dataset({
     features: ['prompt', 'response'],
     num_rows: 20000
 }))

In [ ]:
q72[0]

{'prompt': [{'content': '对下面这段内容进行降重：蓝靛果具有促进消化、补充维生素等作用。蓝靛果中含有大量的天然果酸，特别是柠檬酸和苹果酸等物质的含量特别高，它们被人体所吸收以后，能促进胃部消化液分泌，也能加快肠胃蠕动，提高肠胃的消化能力，对消化不良以及腹部胀痛，都有很好的预防和缓解作用。',
   'content_token_ids': None,
   'country': None,
   'cumulative_logprob': None,
   'finish_reason': None,
   'hashed_ip': '11b5b2edf46678e7662f864335dc9cd488492da2a2edbc265c47a792498f72ae',
   'header': None,
   'language': None,
   'redacted': None,
   'role': 'user',
   'state': None,
   'timestamp': None,
   'toxic': False,
   'turn_identifier': None},
  {'content': '蓝靛果有助于消化和补充维生素。它富含多种天然果酸，如柠檬酸和苹果酸，这些成分在人体内可以促进胃液分泌，加速肠道蠕动，从而增强消化功能，对于消化不良和腹胀等问题有良好的预防和缓解效果。',
   'content_token_ids': None,
   'country': None,
   'cumulative_logprob': None,
   'finish_reason': 'stop',
   'hashed_ip': None,
   'header': None,
   'language': None,
   'redacted': None,
   'role': 'assistant',
   'state': None,
   'timestamp': None,
   'toxic': False,
   'turn_identifier': None}],
 'response': 'Qwen/Qwen2.5-72B-Instruct'}

In [ ]:
def mapq72(example):
    data = example['prompt']
    prompt = data[0]['content']
    response = data[1]['content']
    example['prompt'] = prompt
    example['response'] = response
    return example

In [ ]:
mmlu[0]

{'prompt': {'answer': 1,
  'choices': ['Adams only.', 'Brooks only.', 'Case only.', 'Adams and Brooks'],
  'question': "Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were",
  'subject': ''},
 'response': 0}

In [ ]:
def mapmmlu(example):
    data = example['prompt']
    answer = data['answer']
    choice = data['choices'][answer-1]
    question = data['question']
    example['prompt'] = question
    example['response'] = choice
    return example

In [ ]:
tulu3[0]

{'prompt': 'Alex is a supportive and organized partner who helps their significant other, Jamie, design a system to boost productivity while managing household tasks and work projects. They decide to allocate their time between these tasks using a quadratic model to maximize efficiency. \n\n1. Alex and Jamie determine that the time, in hours, they should spend on household tasks each week can be modeled by the quadratic equation \\( h(t) = -2t^2 + 8t + 5 \\), where \\( t \\) is the number of weeks since they started using the new system. Determine the number of weeks, \\( t \\), after which they should expect to spend the maximum time on household tasks. Also, find the maximum time they should spend on household tasks in a week.\n\n2. To ensure balanced productivity, Alex and Jamie agree to spend at least 15 hours per week on work projects. If the time spent on work projects can be modeled by the inequality \\( w(t) = 3t + 2 \\geq 15 \\), solve for the range of weeks \\( t \\) during w

In [ ]:
def maptulu3(example):
    data = example['response']
    prompt = data[0]['content']
    response = data[1]['content']
    example['prompt'] = prompt
    example['response'] = response
    return example

In [ ]:
q72 = q72.map(mapq72)
mmlu = mmlu.map(mapmmlu)
tulu3 = tulu3.map(maptulu3)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/99842 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
q72[0]

{'prompt': '对下面这段内容进行降重：蓝靛果具有促进消化、补充维生素等作用。蓝靛果中含有大量的天然果酸，特别是柠檬酸和苹果酸等物质的含量特别高，它们被人体所吸收以后，能促进胃部消化液分泌，也能加快肠胃蠕动，提高肠胃的消化能力，对消化不良以及腹部胀痛，都有很好的预防和缓解作用。',
 'response': '蓝靛果有助于消化和补充维生素。它富含多种天然果酸，如柠檬酸和苹果酸，这些成分在人体内可以促进胃液分泌，加速肠道蠕动，从而增强消化功能，对于消化不良和腹胀等问题有良好的预防和缓解效果。'}

In [ ]:
mmlu[0]

{'prompt': "Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were",
 'response': 'Adams only.'}

In [ ]:
tulu3[0]

{'prompt': 'Alex is a supportive and organized partner who helps their significant other, Jamie, design a system to boost productivity while managing household tasks and work projects. They decide to allocate their time between these tasks using a quadratic model to maximize efficiency. \n\n1. Alex and Jamie determine that the time, in hours, they should spend on household tasks each week can be modeled by the quadratic equation \\( h(t) = -2t^2 + 8t + 5 \\), where \\( t \\) is the number of weeks since they started using the new system. Determine the number of weeks, \\( t \\), after which they should expect to spend the maximum time on household tasks. Also, find the maximum time they should spend on household tasks in a week.\n\n2. To ensure balanced productivity, Alex and Jamie agree to spend at least 15 hours per week on work projects. If the time spent on work projects can be modeled by the inequality \\( w(t) = 3t + 2 \\geq 15 \\), solve for the range of weeks \\( t \\) during w

In [ ]:
rewild = concatenate_datasets([q72, mmlu, tulu3])

In [ ]:
rewild[42069]

{'prompt': 'Rewrite this to give it more power in emotional tones:',
 'response': "Certainly! Could you please provide the specific text you'd like me to rewrite?"}

Login to write token

In [ ]:
login()

In [ ]:
rewild.push_to_hub("chardizard/modified-rewild")

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/107 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/107 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/107 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chardizard/modified-rewild/commit/ad47b4f9dfacefbcd612d3093bc72b7e38afef85', commit_message='Upload dataset', commit_description='', oid='ad47b4f9dfacefbcd612d3093bc72b7e38afef85', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/chardizard/modified-rewild', endpoint='https://huggingface.co', repo_type='dataset', repo_id='chardizard/modified-rewild'), pr_revision=None, pr_num=None)